In [ ]:
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
import os
import wandb

# Yêu cầu người dùng nhập Wandb API key
wandb_api_key = input("Nhập Wandb API key của bạn: ")

# Thiết lập biến môi trường WANDB_API_KEY
os.environ["WANDB_API_KEY"] = wandb_api_key

# Đăng nhập vào wandb
wandb.login()

In [ ]:
wandb.init(
    project="IELTS_Slow_Module",  # Tên dự án trên WandB
    name="Run_4"  # Tên cho lần chạy này
)

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

#model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "unsloth/Meta-Llama-3.1-8B",
#    model_name = "unsloth/Llama-3.2-3B",
#    max_seq_length = max_seq_length,
#    dtype = dtype,
#    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
#)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/kaggle/input/ielts_slow_module_checkpoint_500/pytorch/0.3/1/checkpoint-1700",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 24,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Loading libraries

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import pandas as pd
from datasets import Dataset

In [ ]:
from huggingface_hub import login

# Yêu cầu nhập token từ người dùng
token = input("Nhập token Hugging Face của bạn: ")

# Thực hiện đăng nhập
login(token)

In [ ]:
data = pd.read_csv('/kaggle/input/ielts-dataset-full/slow_module_dataset_5.csv')
print(len(data))

In [ ]:
PROMPT = """
In this task, you are required to evaluate an IELTS Writing Task 2 essay. Consider the following criterias and provide a detailed assessment for each:

- Task Achievement:
+ Evaluate if the candidate fully addresses all parts of the task.
+ Assess the clarity of the thesis statement and position.
+ Check if arguments are developed with specific examples/data (e.g., studies, real-world cases).
+ Identify generalizations (e.g., "some people believe") and suggest replacements with evidence.

- Coherence and Cohesion:
+ Assess paragraph structure: Does each paragraph focus on one central idea?
+ Evaluate the variety and accuracy of linking devices (e.g., "however" for contrast, "for instance" for examples).
+ Check if the conclusion logically summarizes key points without introducing new ideas.

- Coherence and Cohesion:
+ Identify repetitive words/phrases and suggest synonyms (e.g., "important" → "crucial," "significant").
+ Highlight unnatural collocations (e.g., "make an interview" → "conduct an interview").
+ Assess the use of topic-specific vocabulary (e.g., "cognitive bias," "behavioral assessment").

- Grammatical Range and Accuracy:
+ List grammatical errors (e.g., subject-verb agreement, tense misuse) and provide corrected versions.
+ Evaluate the use of complex structures (e.g., relative clauses, passive voice).
+ Check punctuation (e.g., comma splices, missing apostrophes).

- Feedback and Additional Comments:
+ Highlight 2–3 strengths (e.g., "effective use of examples") and 3–4 actionable improvements (e.g., "practice using relative clauses").
+ Provide practice suggestions (e.g., "Write 5 sentences using collocations related to employment").

- Prompt:
{}

- Essay:
{}

- Evaluation:
{}
"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    prompt = examples["prompt"]
    essay  = examples["essay"]
    evaluation = examples["evaluation"]
    texts = []
    for prompt, essay, evaluation in zip(prompt, essay, evaluation):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = PROMPT.format(prompt, essay, evaluation) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset_train = Dataset.from_pandas(data)
dataset_train = dataset_train.map(formatting_prompts_func, batched = True,)

In [ ]:
data2 = pd.read_csv('/kaggle/input/ielts-dataset-full/test_slow_module_dataset_4.csv')
data2

In [ ]:
dataset_test = Dataset.from_pandas(data2)
dataset_test = dataset_test.map(formatting_prompts_func, batched = True,)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, TrainerCallback
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_train,
    eval_dataset = dataset_test,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = True, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 10,
        # max_steps = 40,
        learning_rate = 1.75e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 25,
        optim = "adamw_torch",
        weight_decay = 0.05,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "/kaggle/working/ielts_feedback_outputs",
        report_to = "wandb", # Use this for WandB etc
        evaluation_strategy = "steps",  # Đánh giá theo từng step
        eval_steps = 300,  # Điều chỉnh theo nhu cầu
    ),
)

In [ ]:
!pip install timeout-decorator

In [ ]:
import timeout_decorator

@timeout_decorator.timeout(42000)  # 120 giây = 2 phút
def run_training():
    return trainer.train()

try:
    trainer_stats = run_training()
except timeout_decorator.TimeoutError:
    print("Training mất quá 5 phút, chuyển sang cell tiếp theo.")
    trainer.save_model(trainer.args.output_dir)